# Abstract

This project implements machine learning methods to differentiate between music by different composers by inspection of MIDI metadata and audio features predominantly consisting of spectral analysis.

# Dataset Description

This project is based on a public set of classical compositions for piano. The dataset was sourced from http://www.piano-midi.de/. We took the MIDI files as our raw data and used **jAudio** (http://jaudio.sourceforge.net/) to extract features from the MIDI. There are 127 MIDI files (datapoints) in our dataset.

## Attributes

## Data Structuring and Normalization

We limited our data to only include the works of **Chopin**, **Mozart**, **Schubert**, and **Beethoven**. This choice was made to narrow the problem space.

## Splitting the data

# Algorithms

## Gaussian Na&iuml;ve Bayes 

### Implementation Details

### Error On Test Set

### Hyperparameters

## Discrete Na&iuml;ve Bayes

This algorithm uses the metadata and message data directly from the MIDI files to predict the composer of a given piece. The reason for this implementation was to try and perform the classification with discrete data straight from the MIDI files in order to avoid inaccuracies and complications introduced in the processing of the continuous, multi-dimensional features we extracted using jAudio. Although it is not a particularly interesting way to view the data since the composer of a piece is usually specified in the metadata of a MIDI file, however it may provide an interesting contrast to the **Gaussian Bayes** method in terms of results.

### Implementation Details

The features used in this algorithm are extracted directly from the MIDI files and were chosen for simplicity's sake. Due to inconsistent labelling of composer names in the MIDI files, some datapoints fot lost in the data preparation process and thus were not used for this algorithm. The chosen features for this algorithm are as follows:

#### Key Signature

The first key signature given in the MIDI file. Subsequent key signature changes are ignored for simplicity. Key signatures are represented as strings such as `C` or `Ab`. Due to inconsistencies in the representation of key MIDI metadata, some key signatures are represented twice in different ways (for example **F<sup>#</sup>** and **G<sup>b</sup>** which are in fact the same key). This seems to be a strong predictor.

![Key Frequencies](report/plots/key_freq.png)

#### Time Signature

The first time signature given in the MIDI file. Subsequent time signature changes are ignored since they are fairly uncommon in our dataset, and would simply complicate the problem. Time signatures are represented as strings such as `3, 4` or `5, 4`.

#### Mean Tempo

An average of the tempo throughout the whole piece. This is measured in ticks. The mean tempo is then discretized by finding the mean $\mu$ and standard deviation $\sigma^2$ of the mean tempos across all data points and then turning them into discrete values according to the rule:

$T_{class} = 0 \quad$ if $T_{value} < \mu - \sigma^2$, Low tempo.

$T_{class} = 1 \quad$ if $\mu - \sigma^2 <= T_{value} <= \mu + \sigma^2$, Mid tempo.

$T_{class} = 2 \quad$ if $T_{value} < \mu + \sigma^2$, High tempo.

High mean tempos were absent in our dataset.

### Error On Test Set

### Hyperparameters

## Logistic Regression

### Implementation Details

### Error On Test Set

### Hyperparameters

# Discussion of Results

## Best Possible Performance

## Recommendations to Others Working on This Data